# Programme data

This notebook prepares data for the Programme theme page.

In [ ]:
import json
from datetime import date

from utils.themes.programme import Programme
import petl as etl

In [ ]:
from utils.paths import SITE

EVENTS = SITE / 'themes/programme/_data/events'
EVENTS.mkdir(exist_ok=True, parents=True)

## Schedule events

Aggregation definition

In [ ]:
event_aggregation = {
    'Records': ('row', set),
    'Events': ('Event Count', sum),
    'Date From': ('Start Date', min),
    'Date To': ('End Date', max),
}

Read events

In [ ]:
import petl as etl
import pyarrow.parquet as pq
from utils.paths import PUBLISHED

def validation(row):
    if row.project_name is None:
        return 'unknown_project'
    if row.month is None:
        return 'blank_month'
    if row.date > date.today():
        return 'future_dated'
    if row.date < date.fromisoformat('2024-01-01'):
        return 'date_before_2024'
    return None


events_data, invalid_events_data = (
    etl
    .fromdataframe(pq.read_table(PUBLISHED / 'programme/combined.parquet').to_pandas())
    .addfield('validation', validation)
    .biselect(lambda r: r.validation == None)

    # .convert('source', {
    #     'Manual': 1,
    #     'Airtable::Project Hub::Event Reports': 2,
    #     'Airtable::Project Hub::Schedule': 3,
    # })
    # .sort('source')
    # .groupselectfirst(['project_name', 'project_id', 'month', 'variable'])
)

In [ ]:
dimensions = ['project_name', 'project_id', 'month'] 

events = (
    events_data.aggregate(
        [*dimensions, 'variable'],
        {
            'date': ('date', list),
            'start_date': ('start_date', list),
            'end_date': ('end_date', list),
            'value': ('value', sum),
        },
    )
    .recast([*dimensions, 'date', 'start_date', 'end_date'])
    .convert(['date', 'start_date'], min)
    .convert('end_date', max)
)

In [ ]:
events

Read events and break down into chunks broken by month boundaries. This deals with multi-day events which cover more than a single month.

As an example:

| Start -> End | Chunks |
|--------------|--------|
| 15 Jan -> 18 Jan | 1 chunk (15 Jan -> 18 Jan) |
| 15 Jan -> 18 Feb | 2 chunks (15 Jan -> 31 Jan, 1 Feb -> 18 Feb) |
| 15 Jan -> 18 Mar | 3 chunks (15 Jan -> 31 Jan, 1 Feb -> 28 Feb, 1 Mar -> 18 Mar) |

Each chunk is then rendered as a separate row in the table with the `rowmapmany` method.

In [ ]:
monthly_event_reports = (
    Programme.event_reports
    .addfield('Month', lambda r: r.event_date.replace(day=1))
    .aggregate(['Project Name', 'project_id', 'Month'], { 'event_reports': (len), 'event_reports_audience': ('audience', sum) })
)

In [ ]:
monthly_scheduled_events = (
    Programme.events
    .convert(
        'Event Count',
        lambda _, r: ((date.today() - r['Start Date']).days + 1),
        where=lambda r: (r['End Date'] > date.today()),
        pass_row=True
    )
    .aggregate(['Project Name', 'project_id', 'Month'], { 'scheduled_events': ('Event Count', sum) })
)

In [ ]:
monthly_manual_events = (
    Programme.manual_events
    .addfield('Month', lambda r: r.Date.replace(day=1))
    .aggregate(['Project Name', 'Month'], {
        'manual_events': ('Events', sum),
        'manual_audience': ('Audience', sum),
    })
)

In [ ]:
monthly_events = (
    events
    .addfield('events', lambda r: r.manual_events or r.event_reports or r.schedule_events or r.projected_events or 0, index=3)
    .addfield('audience', lambda r: r.event_report_audience or r.manual_audience or 0, index=4)
)

Create an aggregate by month of the events

In [ ]:
(
    monthly_events
    .aggregate('month', {
        'Events': ('events', sum),
        'Audience': ('audience', sum),
        # 'Records': (len),
    })
    .convert('month', lambda f: f.isoformat())
    .tocsv(EVENTS / 'total_by_month.csv')
)

Aggregate by Project and by Month, and convert months to columns

In [ ]:
(
    monthly_events
    .aggregate(['project_name', 'month'], sum, 'events')
    .recast(key='project_name', variablefield='month', missing=0)
    .tocsv(EVENTS / 'monthly_by_project.csv')
)

Aggregate by Project and by Month, and convert projects to columns

In [ ]:
(
    monthly_events
    .aggregate(['project_name', 'month'], sum, 'events')
    .recast(key='month', variablefield='project_name', missing=0)
    .tocsv(EVENTS / 'monthly_breakdown.csv')
)

## Project summaries

In [ ]:
project_breakdown = (
    monthly_events
    .melt(variables=['events', 'event_reports', 'schedule_events', 'projected_events', 'manual_events', 'audience', 'event_report_audience', 'manual_audience'])
    .selectnotnone('value')
    .aggregate(['project_name', 'project_id', 'variable'], {
        'date': ('date', list),
        'start_date': ('start_date', list),
        'end_date': ('end_date', list),
        'value': ('value', sum),
    })
    .recast()
    .convert(['date', 'start_date'], min)
    .convert('end_date', max)
    .leftjoin(Programme.projects.cutout('Project Name'), lkey='project_id', rkey='id')
)

Create a project breakdown

In [ ]:
with open(EVENTS / 'by_project.json', 'w') as f:
    json.dump(
        dict(
            project_breakdown
            .addfield('Details', lambda r: {
                # 'records': r.Records,
                'events': r.events,
                'eventReports': r.event_reports,
                'scheduledEvents': r.schedule_events,
                'projectedEvents': r.projected_events,
                'manual_events': r.manual_events,
                'audience': r.audience,
                'event_reports_audience': r.event_report_audience,
                'manual_audience': r.manual_audience,
                'evaluationCategory': r['Evaluation Category'],
                'programmeCategory': r['Programme Category'],
                'earliestDate': r.start_date.isoformat() if r.start_date else r.date.isoformat() if r.date else None,
                'latestDate': r.end_date.isoformat() if r.end_date else None,
            })
            .cut('project_name', 'Details')
            .sort('project_name')
            .records()
        ),
        f,
        indent=2,
    )

Create a summary file

In [ ]:
with open(EVENTS / 'summary.json', 'w') as f:
    json.dump(
        {
            'total': {
                'events': sum(monthly_events.values('events')),
                'audience': sum(a for a in monthly_events.values('audience') if a is not None)
            },
            'excluded': dict(Programme.excluded_events.aggregate('Validation', len).records()),
            'date': {
                'earliest': min(Programme.events.values('Start Date')).isoformat(),
                'latest': max(Programme.events.values('End Date')).isoformat(),
            }
        },
        f,
        indent=2,
    )

## Venues

In [ ]:
education_settings = (
    Programme.venues
    .selectcontains('Org/Venue Type', 'Education Setting')
)

In [ ]:
loading_bay = (
    Programme.venues
    .selectcontains('Organisation &/or Venue Name', 'Loading Bay')
    .addfield('event_report_count', lambda r: len(r['Event Reports']))
    .aggregate(['Organisation &/or Venue Name', 'id'], sum, 'event_report_count')
    .cutout('id')
)
loading_bay

In [ ]:
beacon = (
    Programme.venues
    .selectcontains('Organisation &/or Venue Name', 'Beacon - ')
    .addfield('event_report_count', lambda r: len(r['Event Reports'] or []))
    .aggregate(['Organisation &/or Venue Name', 'id'], sum, 'event_report_count')
    .cutout('id')
)
beacon

In [ ]:
json.dump(
    {
        'loading_bay': dict(loading_bay.records()),
        'beacon': dict(beacon.records()),
    },
    open(EVENTS / 'by_venue.json', 'w')
)